In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import division
%matplotlib inline

#### 用户信息表  
- test_users.csv  
- train_users_2.csv
- age_gender_bkts.csv

In [ ]:
test = pd.read_csv("../input/test_users.csv")
train = pd.read_csv("../input/train_users_2.csv")
session = pd.read_csv("../input/sessions.csv")
users = pd.concat([test,train],axis=0)
#有缺失值的变量有：
#age, country_destination, date_first_booking,first_affiliate_tracked
users.info()

##### Age  
从注册页面来看，年龄是可选项，且可选的年龄范围跨度120年。因此年龄在120以下的表示的是年龄，120以上的表示的是年份，缺失的表示用户没有填。(实际上有132和150)  
1. 把年份换算成年龄
2. 认为大于100与小于12的，是不准确的值

In [ ]:
users.age = [(2014 - x) if x > 160 else x for x in users.age]
users.loc[users.age>100, "age"] = np.nan
users.loc[users.age<12, "age"] = np.nan
#"NA percentage:'
print sum(pd.isnull(users.age))/users.shape[0]*100

In [ ]:
sns.distplot(users.age.dropna())

In [ ]:
users.country_destination.value_counts().plot.barh()
xlabel('Number of Users')
title('Count of Users by Dest Country')

In [ ]:
users.gender.value_counts().plot.barh()

In [ ]:
users['date_account_created'] = pd.to_datetime(users['date_account_created'])
users['date_first_booking'] = pd.to_datetime(users['date_first_booking'])
users['timestamp_first_active'] = pd.to_datetime(test['timestamp_first_active'], format='%Y%m%d%H%M%S')

In [ ]:
users.affiliate_channel.value_counts().plot.barh()

In [ ]:
users.affiliate_provider.value_counts().plot.barh()

In [ ]:
users.first_affiliate_tracked.value_counts().plot.barh()

In [ ]:
users.date_account_created.value_counts().sort_index().plot()

In [ ]:
bkts = pd.read_csv("../input/age_gender_bkts.csv")
sns.factorplot(x="age_bucket", y="population_in_thousands", hue="gender",col="country_destination",data=bkts,kind="bar",size=6)

In [ ]:
session.apply(lambda x: x.nunique(),axis=0)
grpby = session.groupby(['user_id'])['secs_elapsed'].sum().reset_index()
grpby.columns = ['user_id','secs_elapsed']
session.groupby(['action_type'])['user_id'].nunique().reset_index()